In [3]:
import tkinter as tk
from tkinter import ttk, filedialog, PhotoImage
from tkinter.font import Font
from PIL import Image, ImageTk
import itertools
import matplotlib.pyplot as plt
import cv2
import glob
import numpy as np
import os
from keras.models import load_model
from PIL import ImageSequence
import dlib

# Initialize Dlib's face detector
detector = dlib.get_frontal_face_detector()

    

age_model = load_model('age_recognition.h5')

ranges = [str(age) for age in range(101)]




# Retrieve image file paths from a specified folder
def get_image_paths(folder_path):
    return glob.glob(f'{folder_path}/**/*.jpg', recursive=True)

def preprocess_image(img):
    # Dynamically adjusts image size to maintain aspect ratio
    h, w = img.shape[:2]
    scale = 224 / max(h, w)
    img = cv2.resize(img, (int(w * scale), int(h * scale)))
    return img

# Display both the original and processed images side by side
def display_original_and_processed(original, processed):
    plt.figure(figsize=(10,5))

    plt.subplot(1, 2, 1)
    plt.imshow(original)
    plt.title('Original Image')

    plt.subplot(1, 2, 2)
    plt.imshow(processed)
    plt.title('Processed Image')

    plt.show()

# Process images from the specified folder and return the array of processed images
def process_images_from_folder(folder_path, images_to_display, total_images_to_process=None):
    image_paths = get_image_paths(folder_path)
    
    # If total_images_to_process is specified, limit the number of images to be processed
    if total_images_to_process:
        image_paths = image_paths[:total_images_to_process]
        
    processed_images = []
    
    for index, img_path in enumerate(image_paths):
        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

        processed_img = preprocess_image(img)
        processed_images.append(processed_img)

        # Display images up to the number specified by images_to_display
        if index < images_to_display:
            display_original_and_processed(img, processed_img)

    return np.array(processed_images)

# Function for face detection
def face_detection(pic):
    # Detect faces using Dlib
    dets = detector(pic, 1)
    facesdet = [(d.left(), d.top(), d.right() - d.left(), d.bottom() - d.top()) for d in dets]
    return facesdet

#boxes
# Function to display bounding boxes on detected faces
def box_disp(pic, facesdet, r1, r2, age_predictions):
    for ((x, y, w, h), age_pred) in zip(facesdet, age_predictions):
        # Adjust coordinates
        x1, y1 = int(x * r2), int(y * r1)
        x2, y2 = int(x1 + w * r2), int(y1 + h * r1)

        # Draw rectangle and text
        cv2.rectangle(pic, (x1, y1), (x2, y2), (0, 255, 0), 4)
        cv2.putText(pic, age_pred, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2, cv2.LINE_AA)




def predict_age(cropped_faces):
    age_predictions = []
    for face in cropped_faces:
        face = cv2.resize(face, (60, 60))
        face = face.reshape(1, 60, 60, 1)
        predicted_age = age_model.predict(face)

        if np.argmax(predicted_age) < len(ranges):
            age_range = ranges[np.argmax(predicted_age)]
            age_predictions.append(age_range)
        else:
            print("Index out of range error")
            age_predictions.append("Unknown")
    return age_predictions


# Function to detect faces in a still image and display it
def picCheck(image_path):
    image = cv2.imread(image_path)
    processed_image = preprocess_image(image)
    faces = face_detection(processed_image)
    age_predictions = predict_age(faces)

    scaling_factor_y = image.shape[0] / processed_image.shape[0]
    scaling_factor_x = image.shape[1] / processed_image.shape[1]

    adjusted_faces = [(int(x * scaling_factor_x), int(y * scaling_factor_y), int(w * scaling_factor_x), int(h * scaling_factor_y)) for x, y, w, h in faces]

    box_disp(image, adjusted_faces, 1, 1, age_predictions)

    cv2.imshow("Age Recognition", image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()




# Modified process_video function with age recognition
def process_video(video_path=None):
    cap = cv2.VideoCapture(0 if video_path is None else video_path)

    if not cap.isOpened():
        print("Error: Couldn't open the video stream or file.")
        return

    window_name = 'Age Recognition in Video'
    cv2.namedWindow(window_name, cv2.WINDOW_NORMAL)

    try:
        while cap.isOpened():
            if cv2.getWindowProperty(window_name, cv2.WND_PROP_VISIBLE) < 1:
                break
            ret, frame = cap.read()
            if not ret:
                print("Error: Couldn't read a frame from the video.")
                break

            original_height, original_width = frame.shape[:2]
            processed_frame = preprocess_image(frame)
            scaling_factor_y = original_height / processed_frame.shape[0]
            scaling_factor_x = original_width / processed_frame.shape[1]

            # Detect faces and predict age
            faces = face_detection(processed_frame)
            age_predictions = predict_age(faces)

            # Adjust face coordinates based on the original frame size and display age
            adjusted_faces = [(int(x * scaling_factor_x), int(y * scaling_factor_y), int(w * scaling_factor_x), int(h * scaling_factor_y)) for x, y, w, h in faces]
            box_disp(frame, adjusted_faces, 1, 1, age_predictions)

            cv2.imshow(window_name, frame)

            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
    finally:
        cap.release()
        cv2.destroyAllWindows()


# Functions to open a file dialog for selecting an image/video file
 
def select_picture():
    file_path = filedialog.askopenfilename(filetypes=[("Image Files", "*.jpg;*.jpeg;*.png")])
    if file_path:
        picCheck(file_path)

def select_video():
    file_path = filedialog.askopenfilename(filetypes=[("Video Files", "*.mp4;*.mkv")])
    if file_path:
        process_video(file_path)

def load_image(path):
    img = Image.open(path)
    img = img.resize((100, 100), Image.ANTIALIAS)  # Resize if needed
    return ImageTk.PhotoImage(img)




def create_ui():
    window = tk.Tk()
    window.title("Age Recognition Application")
    window.geometry("1000x700")  # Adjust size as per your requirement
    window.configure(bg='black')

     # Heading
    heading_label = tk.Label(window, text="Age Recognition Application", bg='black', fg='white', font=("Arial", 24))
    heading_label.pack(pady=20)  # Add some padding for the heading

    # Frames for each image and button
    frame1 = tk.Frame(window, bg='black')
    frame1.pack(side=tk.LEFT, padx=10, pady=10, fill='both', expand=True)

    frame2 = tk.Frame(window, bg='black')
    frame2.pack(side=tk.LEFT, padx=10, pady=10, fill='both', expand=True)

    frame3 = tk.Frame(window, bg='black')
    frame3.pack(side=tk.LEFT, padx=10, pady=10, fill='both', expand=True)


    # Function to update the GIF frame
    def update_gif(ind):
        frame = gif_frames[ind]
        ind += 1
        if ind == len(gif_frames):
            ind = 0
        label2.configure(image=frame)
        window.after(100, update_gif, ind)  # Adjust delay as needed

     # Load static images for the first and third frames
    image1 = Image.open('C:\\Users\\mevin\\Desktop\\Untitled Folder 1\\ML--Age-Detection\\pics\\Mino-Tamby-Profile-Picture.jpg').resize((250, 250))
    photo_image1 = ImageTk.PhotoImage(image1)
    label1 = tk.Label(frame1, image=photo_image1)
    label1.image = photo_image1  # Keep a reference!
    label1.pack(pady=10)

    image3 = Image.open('C:\\Users\\mevin\\Desktop\\Untitled Folder 1\\ML--Age-Detection\\pics\\photo-camera-icons-photo-camera-icon-design-illustration-photo-camera-simple-sign-photo-camera-logo-vector.jpg').resize((250, 250))
    photo_image3 = ImageTk.PhotoImage(image3)
    label3 = tk.Label(frame3, image=photo_image3)
    label3.image = photo_image3  # Keep a reference!
    label3.pack(pady=10)

    # Load the GIF for the second frame and create an iterator for its frames
    gif = Image.open('C:\\Users\\mevin\\Desktop\\Untitled Folder 1\\ML--Age-Detection\\pics\\gify.gif')
    gif_frames = [ImageTk.PhotoImage(image.resize((250, 250))) for image in ImageSequence.Iterator(gif)]
    label2 = tk.Label(frame2, image=gif_frames[0])
    label2.image = gif_frames  # Keep a reference!
    label2.pack(pady=10)

    # Start the animation
    window.after(0, update_gif, 0)

    # Pack the labels centered in their frame
    label1.pack(side='top')  # Center the label within the top part of the frame
    label2.pack(side='top')
    label3.pack(side='top')

    # Buttons below images
    button_texts = ['Select Picture', 'Select Video', 'Real-time Check']
    commands = [select_picture, select_video, process_video]
    buttons = [tk.Button(frame, text=text, command=cmd, bg='#4caf50', fg='white') for frame, text, cmd in zip([frame1, frame2, frame3], button_texts, commands)]
    for button in buttons:
        button.pack(fill='x') 

    window.mainloop()

create_ui()

1/1 [==============================] - 0s 27ms/step


In [12]:
pip install dlib

  Using cached dlib-19.24.2-cp311-cp311-win_amd64.whl
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip
